# Run Seqeunce 

`Module 1` 
- Re-format IDT files  <br>

`Module 2` 
- Add water to primers/guides, create picklist for TECAN Fluent
- Create picklist for tarnsfer to 384well plate for PCR setup.

`Module 3` 
- Guides_Picklist`<br>

`Module 4`
- PCR_Picklist

`Module 5`
- Equimolar mixing of PCR fragments

Each module is depependent on successful run of the previous module. 

# Module 3   Guides_Worklist

**This module Generates Picklist for setting up guide phosphorylation **

> `Guide_Phosphorylation_Picklist`

For Each input file belonging to each plasmid, it will generate another file with `Pos_`. <br>
> This file will be written in a subfolder `Position_files` <br>
- This file contains the information for the position of primers/guides in the 96-well plate. <br>
- Since, both primers and guides are in different plates, there will be repeated well positions. <br>
- Remember, primers and guides are in separate 96-well or 384-well plates.

- Create a .csv file `edited_IDT_combined`

    It doesn't matter if the guides were received in 96-well plate or 384-well plate. 
    The script will be same for both situations, unlike the PCR setup. 
    Remember to put in correct plate type in Fluent for guide transfers. 
    
    
-  `edited_IDT_combined.csv` file contains the original positions as found in the IDT plate received from the company. <br>

-  We will set up the guide phopshorylation in a 96-well plate. <br>
-   `Picklist_Guide_Mixing.csv` :- Picklist to mix all guides in a 96-well plate

Consolidate the guides to one file, information about what wells conatins the guide for what plasmids.

- Create the Picklist for PNK master mix distribution

`Picklist_PNK_Guides_phospho.csv` Contains info for:
-  Transfer 10ul of mixed guides to next wells. 
-  Add PNK master mix 10ul to each well tha conatins 10ul guide mix
-  The position of PNK master mix needs to be marked


In [1]:
import os, sys
import traceback
import pandas as pd
import re
import math
import numpy as np
import glob
from string import ascii_uppercase
from natsort import natsorted


def write_to_subfolder(filename, dataframe_object, subdirectory):
    
    curr_dir = os.getcwd()
    new_dir = os.path.join(curr_dir,subdirectory)
    counter = 0
    
    if curr_dir.find(subdirectory) == -1:
        
        try:
            os.mkdir(subdirectory)
        except Exception:
            pass
    
        os.chdir(new_dir)
        print("File written :-   ", filename)
        dataframe_object.to_csv(filename, index=False)
        counter = 1
        os.chdir('..')
        
    else:
        os.chdir(new_dir)
        print("File written :-   ", filename)
        dataframe_object.to_csv(filename, index=False)
        os.chdir('..')


def Combine_Primer_Guide_Files():
    
    counter = 0 
    df_all = pd.DataFrame()
    
    y = glob.glob("**/*IDT_Primers_*")
    x = glob.glob("**/*IDT_Guides_*")

    print(y[0]+' and '+x[0]+' files combined. \n ')
    df_temp  = pd.read_csv(y[0])
    df_all = df_all.append(df_temp)
    df_temp  = pd.read_csv(x[0])
    df_all = df_all.append(df_temp)
    df_all= df_all.reset_index(drop=True)
  
    write_to_subfolder('edited_IDT_combined.csv', df_all, 'Generated_Files')
    
    Prepare_Guide_Mixing_Picklist(df_all)
    
    
def Define_Wellplate96():
    
    Wellplate96 = []
    for i in range(65,73):
        for j in range(1,13):
            dest = chr(i)+str(j)
            Wellplate96.append(dest)
            
    return Wellplate96
    

def Prepare_Guide_Mixing_Picklist(df_IDT):
        
    file_list = glob.glob("Primers_Guides_neb*.csv")
    file_list = natsorted(file_list)
    print("Number of plasmids :- ", len(file_list), '\n')
    
    Wellplate96 = Define_Wellplate96()
       
    counter = 0       
    Guide_df = pd.DataFrame()
    for filename in file_list:
        df_original = Mark_well_positions(filename, df_IDT)
        
        Dest_well = Wellplate96[counter]
        counter = counter + 1
        Temp_df = create_guide_picklist(df_original, Dest_well, filename)
        Guide_df = Guide_df.append(Temp_df)
        
    Guide_df = Guide_df.reset_index(drop =True)
    Guide_df = Fluent_worklist(filename, Guide_df)
    Guide_df = Guide_df.reset_index(drop =True)

    filename  = 'Picklist_Guide_Mixing.csv'
    print('\n')
    write_to_subfolder(filename, Guide_df, 'Dilution_Picklist')
    Consolidate_Guides(Guide_df)
    
    
def Consolidate_Guides(Guide_df):
        
    #Guides_new_df = Guide_df.drop(['Source Well', 'Volume'], axis =1)
    Guides_new_df = Guide_df[Guide_df.Plasmid_Name != '']
    #filename_new = 'Guides_phospho_consolidated.csv'
    #write_to_subfolder(filename_new, Guides_new_df, 'Dilution_Picklist')
    Guides_new_df = Guides_new_df.reset_index(drop =True)
    Guide_mixing_and_phosphorylation(Guides_new_df)
    
    
def Fluent_worklist(Filename, Guide_df):
    
    Guide_wells = Guide_df['Source Well'].tolist()
    
    if 'A13' in Guide_wells:
        Dest_plate = 'Guides384well'
    else:
        Dest_plate = 'Guides96Deepwell'
        
    Guide_df['Source_plate'] = Dest_plate
    Guide_df['Dest_plate'] = 'GuideMix96well'
    
    cols = ['Source_plate', 'Source Well', 'Dest_plate', 'Destination Well', 'Volume', 'Plasmid_Name' ]
    Guide_df = Guide_df[cols]
    
    return Guide_df
    

    
    
def create_guide_picklist(df_original, Dest_well, plasmid_name):

    column_names = ["Source Well", "Destination Well", "Volume", "Plasmid_Name"]
    df_picklist = pd.DataFrame(columns = column_names)
    list2 = []
    counter3 = 0
    Vol = 10
    
    for index, row in df_original.iterrows():
        Name = df_original['Name'].loc[index]
        isGuide = re.search("[G][0-9]\S[j][0-9]", Name)
        
        if isGuide:
            s_well = df_original['IDT_Well_Position'].loc[index]
            #d_well = Dest_well
            p_name = ''
            if counter3 == 0:
                p_name = plasmid_name[19:-4]
                counter3 = counter3 + 1
            
            list2 = [s_well, Dest_well, Vol, p_name]
            a_series = pd.Series(list2, index = df_picklist.columns)
            df_picklist = df_picklist.append(a_series, ignore_index=True)
            
        else:
            continue
    df_picklist = df_picklist.reset_index(drop=True)
    
    return df_picklist


def Mark_well_positions (Original_filename, df_IDT):
    
    df_original = pd.read_csv(Original_filename)
    df_original  = df_original.drop(['Scale', 'Purification'], axis=1)

    list2 = []

    for index, row in df_original.iterrows():
        
        Sequence = str(df_original['Sequence'].loc[index])
        Match_well = df_IDT[df_IDT['Sequence'].str.match(Sequence)]
                  
        counter2 = 0
        for index2, row2 in Match_well.iterrows():
            if counter2 == 0:
                well_pos  = Match_well['Well Position'].loc[index2]
                Seq2 = str(Match_well['Sequence'].loc[index2])
                if Sequence == Seq2:
                    list2.append(well_pos)
                    counter2 = counter2 + 1
                                        
    subdirectory = 'Position_files'
    df_original['IDT_Well_Position'] = list2
    df_original = df_original.reset_index(drop= True)

    file_2 = 'Pos_'+Original_filename[:-4]+'.csv'
    write_to_subfolder(file_2, df_original, subdirectory)
    
    return df_original

def Assign_master_mix_location(n, wellname):
    
    print('\n Any well after the well {}, Enter in format "B11"..   '.format(wellname))
    print('\n No validation of well names, enter correct format \n ')
    
    master_mix_location = []
    for i in range(n):
        Well_1 = input('\t Enter well name {} :- '.format(i+1))
        master_mix_location.append(Well_1)
        
    return master_mix_location
    
    
def Guide_mixing_and_phosphorylation(Temp_df):

    column_names = ['Source_Well', 'Destination_well', 'Volume', 'Plasmid_Name']
    Transfer10ul_df = pd.DataFrame(columns = column_names)
    Master_mix_df = pd.DataFrame(columns = column_names)
    
    Wellplate96 = Define_Wellplate96()
        
    counter = 0
    rows = math.ceil(len(Temp_df)/12)
    first_index = 0

    print('\n Minimumn number of wells with master mix :-  ', math.ceil(len(Temp_df)/10))
    print("\n Master mix volume required > ", 10*(len(Temp_df)+1))
    n = math.ceil(len(Temp_df)/10)
    master_mix_location = Assign_master_mix_location(n, Wellplate96[first_index])
    
    list1 = []
    list2 = []
    volume = 10
    master_mix = 'Master mix'
    print('\n No of plamids for guide mixing: ', len(Temp_df))
    master_mix_location_list = []
    
    k = -1
    for i in range(len(Temp_df)):
        if i%10 == 0:
            k = k + 1
        master_mix_location_list.append(master_mix_location[k])
        
    for index, row in Temp_df.iterrows():
        source_Well = Temp_df['Destination Well'].loc[index]
        dest_well = Wellplate96[first_index]
        first_index = first_index + 1
        plasmid = Temp_df['Plasmid_Name'].loc[index]
        
        list1 = [source_Well, dest_well, volume, plasmid]
        a_series = pd.Series(list1, index = Transfer10ul_df.columns)
        Transfer10ul_df = Transfer10ul_df.append(a_series, ignore_index=True)
        
    Master_mix_df = Transfer10ul_df
    
    Master_mix_df = Master_mix_df.assign(Source_Well=master_mix_location_list)
    Master_mix_df = Master_mix_df.assign(Plasmid_Name='')        
        
    Transfer10ul_df = Transfer10ul_df.append(Master_mix_df)
    Transfer10ul_df = Transfer10ul_df.reset_index(drop = True)
    filename = 'Picklist_PNK_Guides_phospho.csv'
    
    write_to_subfolder(filename, Transfer10ul_df, 'Dilution_Picklist')
    print(Transfer10ul_df.to_string())
    
    
if __name__ == '__main__':
    
    print(os.getcwd(), '\n')
    Combine_Primer_Guide_Files()

/Users/nilmani/Desktop/Python/High_GC_Picklist 

Generated_Files/IDT_Primers_highGC.csv and Generated_Files/IDT_Guides_highGC.csv files combined. 
 
File written :-    edited_IDT_combined.csv
Number of plasmids :-  5 

File written :-    Pos_Primers_Guides_neb_1GC.csv
File written :-    Pos_Primers_Guides_neb_2GC.csv
File written :-    Pos_Primers_Guides_neb_3GC.csv
File written :-    Pos_Primers_Guides_neb_4GC.csv
File written :-    Pos_Primers_Guides_neb_5GC.csv


File written :-    Picklist_Guide_Mixing.csv

 Minimumn number of wells with master mix :-   1

 Master mix volume required >  60

 Any well after the well A1, Enter in format "B11"..   

 No validation of well names, enter correct format 
 


	 Enter well name 1 :-  A12



 No of plamids for guide mixing:  5
File written :-    Picklist_PNK_Guides_phospho.csv
  Source_Well Destination_well Volume Plasmid_Name
0          A1               A1     10          1GC
1          A2               A2     10          2GC
2          A3               A3     10          3GC
3          A4               A4     10          4GC
4          A5               A5     10          5GC
5         A12               A1     10             
6         A12               A2     10             
7         A12               A3     10             
8         A12               A4     10             
9         A12               A5     10             


### Verify that all position files have an associated well name

Once, we have written the position files, we want to be sure that all primers and guides have their well position marked. 

In [2]:
def read_all_pos_files():
    
    file_list = glob.glob("**/*Pos_Primers_Guides_*.csv")
    file_list = natsorted(file_list)

    for filename in file_list:
        verify_position(filename)
        
def verify_position(filename):
    
    df = pd.read_csv(filename)
    flag = -1
    
    for index, row in df.iterrows():
        
        Name = df['Name'].loc[index]
        well = df['IDT_Well_Position'].loc[index]
        
        if well:
            continue
        else:
            flag = 1
            print('MISSING!!!!! well positions in', filename, Name)
            
    if flag < 0:
        print("No missed well labeling in --> ", filename)

    
if __name__ == '__main__':
    
    read_all_pos_files()
    
    

No missed well labeling in -->  Position_files/Pos_Primers_Guides_neb_1GC.csv
No missed well labeling in -->  Position_files/Pos_Primers_Guides_neb_2GC.csv
No missed well labeling in -->  Position_files/Pos_Primers_Guides_neb_3GC.csv
No missed well labeling in -->  Position_files/Pos_Primers_Guides_neb_4GC.csv
No missed well labeling in -->  Position_files/Pos_Primers_Guides_neb_5GC.csv


## THE END



In [ ]:
def Consolidate_Guides(Guide_df):
    # In case, this one is ever needed    
    Guides_new_df = Guide_df.drop(['Source Well', 'Volume'], axis =1)
    Guides_new_df = Guides_new_df[Guides_new_df.Plasmid_Name != '']
    filename_new = 'Guides_phospho_consolidated.csv'
    write_to_subfolder(filename_new, Guides_new_df, 'Dilution_Picklist')
    Guides_new_df = Guides_new_df.reset_index(drop =True)
    Guide_mixing_and_phosphorylation(Guides_new_df)